# Atari

### Import libraries

In [1]:
import sys
sys.path

['',
 '/home/ricardo/anaconda2_5.1/envs/rl/lib/python27.zip',
 '/home/ricardo/anaconda2_5.1/envs/rl/lib/python2.7',
 '/home/ricardo/anaconda2_5.1/envs/rl/lib/python2.7/plat-linux2',
 '/home/ricardo/anaconda2_5.1/envs/rl/lib/python2.7/lib-tk',
 '/home/ricardo/anaconda2_5.1/envs/rl/lib/python2.7/lib-old',
 '/home/ricardo/anaconda2_5.1/envs/rl/lib/python2.7/lib-dynload',
 '/home/ricardo/anaconda2_5.1/envs/rl/lib/python2.7/site-packages',
 '/home/ricardo/anaconda2_5.1/envs/rl/lib/python2.7/site-packages/IPython/extensions',
 '/home/ricardo/.ipython']

In [2]:
%matplotlib inline

import os
import h5py
import gym
import numpy as np
import pandas as pd
import random
from time import sleep
from numpy.random import randint
from collections import deque


from keras.initializers import normal, identity

from keras.models import load_model
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD , Adam, Adagrad, RMSprop
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Dense, Merge
from keras.models import Model


from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

/home/ricardo/anaconda2_5.1/envs/rl/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Creating the game environment

In [3]:
GAME = 'Breakout-ram-v0'
aux = gym.make(GAME)


### Default parameters

In [4]:
# can be configured
BATCH_SIZE = 10
MAX_ITERATIONS_PER_EPISODE = 200
LEARNING_RATE = 0.00025
TARGET_UPDATE_LIMIT = 1
EPSILON = 1.0
EPSILON_UPDATE = 0.1
MIN_EPSILON = 0.1
GAMMA = 0.95
TERMINAL_REWARD = 0
NUMBER_OF_FRAMES = 1

# cannot be configured
INPUT_SIZE = aux.observation_space.shape[0]
OUTPUT_SIZE = aux.action_space.n
MIN_EXPERIENCE_REPLAY_SIZE = 1000
MAX_EXPERIENCE_REPLAY_SIZE = 100000
DATA_TYPE = np.uint8

In [5]:
# np.concatenate(dqn.last_states).shape

In [6]:
# len(dqn.replay)

### Random positions for average q-value

In [7]:
SIZE=10000

random_states_filename = 'random_states_breakout.h5'
if not os.path.isfile(random_states_filename):
    print 'Generating random states'
    with h5py.File(random_states_filename, 'w') as h5:
        random_states = h5.create_dataset('random_states', (SIZE, NUMBER_OF_FRAMES, INPUT_SIZE), dtype=DATA_TYPE)
        data = deque()
        data.append(aux.reset())
        done = False
        for i in range(0, SIZE):
            if done:
                data.clear()
                state = aux.reset()
                done = False
            else:
                state, _, done, _ = aux.step(aux.action_space.sample())                
            
            data.append(state)
            if (len(data) > NUMBER_OF_FRAMES):
                data.popleft()
            if (len(data) == NUMBER_OF_FRAMES):
                random_states[i] = state
else:
    print 'Random states already exists'
            
random_states = np.zeros((SIZE, NUMBER_OF_FRAMES * INPUT_SIZE), dtype=DATA_TYPE)
with h5py.File(random_states_filename, 'r') as h5:
    X = h5.get('random_states')
    for i in range(0, SIZE):
        for j in range(0, NUMBER_OF_FRAMES):
            random_states[i] = np.concatenate(X[i])
            
random_states = random_states / 256.

Random states already exists


In [8]:
random_states.shape

(10000, 128)

# Main source code

In [17]:
def build_model(learning_rate):
    """"Return the neural network"""        
    model = Sequential()
    model.add(Dense(1024, kernel_initializer='he_normal', activation='relu', input_dim=(INPUT_SIZE * NUMBER_OF_FRAMES)))
    model.add(BatchNormalization())
    model.add(Dense(1024, kernel_initializer='he_normal', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(OUTPUT_SIZE, kernel_initializer='truncated_normal'))

    optimizer = SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

class DeepQNetwork:
    
    def __init__(self, **kwargs):       
        if 'batch_size' in kwargs:
            self.batch_size = kwargs['batch_size']
        else:
            self.batch_size = BATCH_SIZE

        if 'min_experience_replay_size' in kwargs:
            self.minimum_experience_replay_size = kwargs['min_experience_replay_size']
        else:
            self.minimum_experience_replay_size = MIN_EXPERIENCE_REPLAY_SIZE
            
        if 'learning_rate' in kwargs:
            self.learning_rate = kwargs['learning_rate']
        else:
            self.learning_rate = LEARNING_RATE

        if 'epsilon' in kwargs:
            self.epsilon = kwargs['epsilon']
        else:
            self.epsilon = EPSILON
        
        if 'gamma' in kwargs:
            self.gamma = kwargs['gamma']
        else:
            self.gamma = GAMMA
            
        if 'target_update_limit' in kwargs:
            self.target_update_limit = kwargs['target_update_limit']
        else:
            self.target_update_limit = TARGET_UPDATE_LIMIT
            
        if 'max_iterations_per_episode' in kwargs:
            self.max_iterations_per_episode = kwargs['max_iterations_per_episode']
        else:
            self.max_iterations_per_episode = MAX_ITERATIONS_PER_EPISODE
            
        if 'min_epsilon' in kwargs:
            self.min_epsilon = kwargs['min_epsilon']
        else:
            self.min_epsilon = MIN_EPSILON
            
        if 'terminal_reward' in kwargs:
            self.terminal_reward = kwargs['terminal_reward']
        else:
            self.terminal_reward = TERMINAL_REWARD
            
        if 'epsilon_update' in kwargs:
            self.epsilon_update = kwargs['epsilon_update']
        else:
            self.epsilon_update = EPSILON_UPDATE
        
        self.last_action = None
        self.last_states = deque()
        self.total_last_states = 0
        
        self.replay = deque()
        self.total_replay = 0
        
        self.env = gym.make(GAME)
        
        self.reward_sum = 0
        self.total_episodes = 0
        
        self.episode_iterations = 0
        self.target_update = 0                
        
        self.model = build_model(self.learning_rate)
        self.frozen_model = build_model(self.learning_rate)
        
        
    def choose_best_action(self):
        """Return the action a that maximizes q(self.last_states, a)"""
        s = np.array([np.concatenate([self.last_states]).reshape(NUMBER_OF_FRAMES * INPUT_SIZE)])
        q = self.model.predict(s/256.)[0]
        action = np.argmax(q)
        return action, q[action]

    
    def choose_random_action(self):
        return self.env.action_space.sample()
    
    
    def choose_e_greedy_action(self):
        """Return an action chosen following the e-greedy policy"""
        if random.random() <= self.epsilon or self.total_last_states < NUMBER_OF_FRAMES:
            return self.choose_random_action()
        else:
            action, _ = self.choose_best_action()
            return action
        
        
    def execute_action(self, action):
        """Return the reward for executing the action and a boolean
           indicating if the new state is terminal.
        
        """
        state, original_reward, done, _ = self.env.step(action)        
        reward = original_reward
        self.reward_sum += reward
        
#         if done:
#             reward = self.terminal_reward
        
        self.add_replay(action, reward, state, done)        
        
        self.episode_iterations += 1
        if done or self.episode_iterations >= self.max_iterations_per_episode:
            self.episode_iterations = 0
            done = True
            self.total_episodes += 1
            self.reset_states()
        else:
            self.add_state(state)
                
        return original_reward, done
    
    
    def add_replay(self, action, reward, state, done):
        previous_state = None
        if self.total_last_states == NUMBER_OF_FRAMES:
            previous_state = self.copy_last_states()
            
        self.add_state(state)
            
        if previous_state is not None:
            current_state = self.copy_last_states()            
            self.replay.append((previous_state, action, reward, current_state, done))
            self.total_replay += 1
            if self.total_replay > MAX_EXPERIENCE_REPLAY_SIZE:
                self.replay.popleft()      
                self.total_replay -= 1

    
    def copy_last_states(self):
        v = []
        for s in self.last_states:
            v.append(np.copy(s))
        return v
    
    
    def add_state(self, state):
        self.last_states.append(np.array(state, dtype=DATA_TYPE))
        self.total_last_states += 1
        if self.total_last_states > NUMBER_OF_FRAMES:
            self.last_states.popleft()
            self.total_last_states -= 1
            
            
    def reset_states(self):
        self.last_states.clear()
        self.last_states.append(np.array(self.env.reset(), dtype=DATA_TYPE))
        self.total_last_states = 1            
        
    def run_test_average_reward(self, total_episodes, render): 
        """Run the environment without traning.
        
           Keyword arguments:
           total_episodes -- number of times the environment 
               will be run.
           render -- boolean indicating if the screen must be
               rendered
        """
        reward_sum = 0
        total_iterations = 0
                
        for _ in range(0, total_episodes):
            self.episode_iterations = 0
            self.reset_states()
            done = False
            reward = 0
            
            while not done:
                if render:
                    sleep(0.03)
                    self.env.render()
                
                if self.total_last_states < NUMBER_OF_FRAMES:
                    action = self.choose_random_action()
                else:  
#                     print self.total_last_states
#                     print self.last_states.shape
                    action, _ = self.choose_best_action()
                total_iterations += 1
                
                r, done = self.execute_action(action)
                reward += r              
            reward_sum += reward            
            
        avg_reward = reward_sum / float(total_iterations)
        
        return avg_reward
    
    
    def run_test_average_qvalue(self):
        """Calculate the average max q-value for the random_states"""
        y = self.model.predict(random_states)        
        return np.average(np.amax(y, axis=1))
        
    def update_network(self):
        """Execute a mini-batch update
        """
        batch = random.sample(self.replay, self.batch_size - 1)
        batch.append(self.replay[-1])

        X_last = np.zeros((self.batch_size, NUMBER_OF_FRAMES * INPUT_SIZE), dtype=DATA_TYPE)
        X_current = np.zeros((self.batch_size, NUMBER_OF_FRAMES * INPUT_SIZE), dtype=DATA_TYPE)        

        for i in range(0, self.batch_size):
            ls, la, r, s, d = batch[i]
            X_last[i] = np.concatenate([ls]).reshape(NUMBER_OF_FRAMES * INPUT_SIZE)
            X_current[i] = np.concatenate([s]).reshape(NUMBER_OF_FRAMES * INPUT_SIZE)
            
        y = self.model.predict(X_last/256.)
        
        self.target_update += 1        
        if self.target_update >= self.target_update_limit:
            self.target_update = 0
            self.frozen_model.set_weights(self.model.get_weights())
            
        q_theta = self.frozen_model.predict(X_current/256.)
        
        for i in range(0, self.batch_size):
            _, la, r, _, d = batch[i]

            if d:
                score = r
            else:
                score = r + self.gamma * np.max(q_theta[i])

            y[i][la] = score

        loss = self.model.train_on_batch(X_last/256., y)
#         loss = self.model.fit(X_last, y, batch_size=32, nb_epoch=3, verbose=1)        
            
    def train(self, total_frames, render):
        """Run the neural network training
        
           Keyword arguments:
           total_frames -- number of times the training process will be executed
           render -- if the screen should be rendered
           
        """
        self.reset_states()
               
        training_iterations = 0
        
        self.episode_iterations = 0
        while(training_iterations < total_frames):
            if render:
                self.env.render()           
        
            action = self.choose_e_greedy_action()
            _, done = self.execute_action(action)
            
            if self.total_replay > MIN_EXPERIENCE_REPLAY_SIZE:
                training_iterations += 1
                self.update_network()
                
            if done:
                self.reset_states()
                
        # update epsilon        
        self.epsilon = max(self.min_epsilon, self.epsilon - self.epsilon_update)

                
class Experiment:
    def __init__(self):
        self.data = []
        self.best_score = None
        
    def add_data(self, epoch, average_reward, average_qvalue):
        t = (epoch, average_reward, average_qvalue)
        print 'epoch: {} avg_reward: {} avg_qvalue: {}'.format(epoch, average_reward, average_qvalue)
        self.data.append(t)
        
        
    def get_dataframe(self):
        df = pd.DataFrame(self.data, columns=['epoch', 'avg_reward', 'avg_qvalue'])
        return df
    
    
    def execute(self, epochs, dqn, model_name):    
        self.best_score = None 
        self.best_qvalue = None
        self.best_filename = None
        c = 0
        
        for i in tqdm(range(epochs)):
            c += 1
            
            filename = '/tmp/%s.%03d.h5'%(model_name, c)
            
            # train
            dqn.train(2000, False)                 

            # test
            avg_reward = dqn.run_test_average_reward(100, False)            
            avg_qvalue = dqn.run_test_average_qvalue()
            if self.best_score is None or avg_reward > self.best_score or \
                (avg_reward == self.best_score and avg_qvalue >= self.best_qvalue):
                    
                self.best_score = avg_reward
                self.best_qvalue = avg_qvalue
                self.best_filename = filename

            self.add_data(c, avg_reward, avg_qvalue)

            # save model            
            dqn.model.save_weights(filename)

            
def generate_graphic(filename, adjusted_qvalue, qvalue_range=[-100,150], reward_range=[0,250]):
    sns.set_context("paper")
    sns.set_style('dark')
    
    df = pd.DataFrame(experiment.get_dataframe(), columns=['epoch', 'avg_reward', 'avg_qvalue'])
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.plot(df['epoch'], df['avg_reward'])
    if reward_range is not None:
        ax1.set_ylim(reward_range)
    ax1.set_xlabel('Number of epochs')
    ax1.set_ylabel('Average reward')
    
    ax1.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
    ax1.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator())
    ax1.grid(b=True, which='major', color='w', linewidth=1.5)
    ax1.grid(b=True, which='minor', color='w', linewidth=0.5)

    ax2 = ax1.twinx()
    ax2.plot(df['epoch'], df['avg_qvalue'] + adjusted_qvalue, color='r')
    ax2.set_ylabel('Average q-value')
    if qvalue_range is not None:
        ax2.set_ylim(qvalue_range)
    
    ax1.legend(loc=2)
    ax2.legend(loc=0)
    
    plt.savefig(filename)

# Experiments

The running timing for most of training is about 4 to 10 minutes, except when the maximum number of iterations in an episode is greater than the default value (200).

### Best model - restricted
- target update 200
- gamma 0.99
- reward at end -100
- learning rate 0.0001

epoch, average_reward, average_qvalue

In [18]:
random_states.shape

(10000, 128)

In [ ]:
label = 'best_restricted'

config = {
    'epsilon': 0.5,
    'gamma' : 0.9,
    'target_update_limit' : 100,
    'learning_rate' : 0.0001,
    'max_iterations_per_episode' : 1000
}

# dqn = DeepQNetwork(**config)
dqn.batch_size = 50

experiment = Experiment()
experiment.execute(500, dqn, label)
print 'Best iteration'
print 'score: {} | qvalue: {} | model: {}' \
    .format(experiment.best_score, experiment.best_qvalue, experiment.best_filename)
generate_graphic('report/images/{}.pdf'.format(label), 0)





  0%|          | 0/500 [00:00<?, ?it/s]



  0%|          | 1/500 [02:48<23:18:40, 168.18s/it]

epoch: 1 avg_reward: 0.0 avg_qvalue: nan






  0%|          | 2/500 [05:35<23:13:58, 167.95s/it]

epoch: 2 avg_reward: 0.0 avg_qvalue: nan






  1%|          | 3/500 [09:00<24:52:53, 180.23s/it]

epoch: 3 avg_reward: 0.0 avg_qvalue: nan


### Best model - unrestricted
- target update 200
- gamma 0.99
- reward at end -100
- max iterations per episode 1000
- learning rate 0.0001

In [ ]:
label = 'best_unrestricted'

config = {
    'terminal_reward' : -100,
    'gamma' : 0.9,
    'target_update_limit' : 200,
    'max_iterations_per_episode' : 1000,
    'learning_rate' : 0.001
}

# dqn = DeepQNetwork(**config)

experiment = Experiment()
experiment.execute(500, dqn, label)
print 'Best iteration'
print 'score: {} | qvalue: {} | model: {}' \
    .format(experiment.best_score, experiment.best_qvalue, experiment.best_filename)
generate_graphic('report/images/{}.pdf'.format(label), 0, None, None)

### Generating the graphic for previous model, setting the y_lim

### Render an episode using best model found - iteration 090 from previous model

In [25]:
config = {
    'max_iterations_per_episode' : 5000000
}

dqn_test = DeepQNetwork(**config)
dqn_test.model.load_weights('boxing.h5')
score = dqn_test.run_test_average_reward(1, False)
print 'Score: {}'.format(score)

Score: 0.00589721988206


In [26]:
dqn = dqn_test

In [55]:
dqn.run_test_average_reward(1000, True)

KeyboardInterrupt: 